In [7]:
my_files=['G:/data/QudiData/2018/02/20180208/PulsedMeasurement/20180208-1854-53_NV3_WaitingSpinLocking_0.1126_NV3_pulsed_measurement.dat',
         'G:/data/QudiData/2018/02/20180208/PulsedMeasurement/20180208-1946-26_NV3_SpinLocking_0.1126_NV3_pulsed_measurement.dat']
contrast=30
N=16

In [8]:
import numpy as np

# for i in range(len(my_files)):
myfile=my_files[0]
f = open(myfile, 'r')
lines = f.readlines()
result = []
for x in lines:
    result.append(x.split('#')[0])
f.close()
a = [x for x in result if x != '']
    
evolution_time = np.zeros(len(a))
counts1 = np.zeros(len(a))
error1 = np.zeros(len(a))
counts2 = np.zeros(len(a))
error2 = np.zeros(len(a))

for i in range(len(a)):
    evolution_time[i]=np.asarray(a[i].split(), dtype=np.float32)[0]
    counts1[i] = np.asarray(a[i].split(), dtype=np.float32)[1]
    error1[i] = np.asarray(a[i].split(), dtype=np.float32)[3]
    counts2[i] = np.asarray(a[i].split(), dtype=np.float32)[2]
    error2[i] = np.asarray(a[i].split(), dtype=np.float32)[4]

baseline = np.sum(counts2+counts1)/len(counts2)/2
C0_up = baseline / (1 - 0.01 * contrast / 2)
C0_down = C0_up * (1 - 0.01 * contrast)
counts = counts2 - counts1

T = evolution_time * 10 * N

normalized_counts = (counts) / (C0_up - C0_down)
freq=0.5/evolution_time/1.0e+6

In [9]:
import numpy as np

# for i in range(len(my_files)):
myfile=my_files[1]
f = open(myfile, 'r')
lines = f.readlines()
result = []
for x in lines:
    result.append(x.split('#')[0])
f.close()
a = [x for x in result if x != '']
    
evolution_time2 = np.zeros(len(a))
counts1 = np.zeros(len(a))
error1 = np.zeros(len(a))
counts2 = np.zeros(len(a))
error2 = np.zeros(len(a))

for i in range(len(a)):
    evolution_time2[i]=np.asarray(a[i].split(), dtype=np.float32)[0]
    counts1[i] = np.asarray(a[i].split(), dtype=np.float32)[1]
    error1[i] = np.asarray(a[i].split(), dtype=np.float32)[3]
    counts2[i] = np.asarray(a[i].split(), dtype=np.float32)[2]
    error2[i] = np.asarray(a[i].split(), dtype=np.float32)[4]

baseline = np.sum(counts2+counts1)/len(counts2)/2
C0_up = baseline / (1 - 0.01 * contrast / 2)
C0_down = C0_up * (1 - 0.01 * contrast)
counts = counts2 - counts1

T = evolution_time * 10 * N

normalized_counts2 = (counts) / (C0_up - C0_down)
freq=0.5/evolution_time/1.0e+6

In [12]:
from plotly import __version__
from plotly.offline import init_notebook_mode, plot
from plotly.graph_objs import Scatter

init_notebook_mode()

trace1 = Scatter(
    x = evolution_time*1e+6,
    y = normalized_counts-normalized_counts2,
    error_y=dict(
            type='data',
            array=error2,
            visible=True
                ),
    mode = 'lines+markers',
    name = 'lines'
)

trace2 = Scatter(
    x = evolution_time*1e+6,
    y = normalized_counts2,
    error_y=dict(
            type='data',
            array=error2,
            visible=True
                ),
    mode = 'lines+markers',
    name = 'just locking'
)
trace3 = Scatter(
    x = evolution_time*1e+6,
    y = normalized_counts,
    error_y=dict(
            type='data',
            array=error2,
            visible=True
                ),
    mode = 'lines+markers',
    name = 'with waiting'
)


plot_data = [trace2, trace3]
plot(plot_data, filename='scatter-mode')

'file://C:\\Users\\pi3\\Documents\\GitHub\\qudi\\scatter-mode.html'

In [31]:
np.savetxt('G:/data/QudiData/2018/01/20180119/PulsedMeasurement/counts_kddxy_6_pol.txt', normalized_counts, delimiter=',')
np.savetxt('G:/data/QudiData/2018/01/20180119/PulsedMeasurement/freq_kddxy_6_pol.txt', freq, delimiter=',')

In [23]:
import pylab as plb
from scipy.optimize import curve_fit
from scipy import asarray as ar,exp

x=0.5/evolution_time/1.0e+6
y=normalized_counts

n = len(x)                          #the number of data
mean = sum(x*y)/n                   #note this correction
sigma = sum(y*(x-mean)**2)/n        #note this correction

def gaus(x,a,x0,sigma):
    return y[0]-a*exp(-(x-x0)**2/(2*sigma**2))

popt,pcov = curve_fit(gaus,x,y,p0=[0.3, 2.4, 0.1])

print(popt)

[ 0.15169233  2.41114487  0.05723856]


In [30]:
from plotly import __version__
from plotly.offline import init_notebook_mode, plot
from plotly.graph_objs import Scatter

init_notebook_mode()

# print("plotly version:", __version__)

freq=0.5/evolution_time/1.0e+6
popt=[ 0.15,  2.41,  0.03]

trace1 = Scatter(
    x = freq,
    y = normalized_counts,
    mode = 'lines+markers',
    name = 'lines+markers'
)
trace2 = Scatter(
    x = np.linspace(freq.max(), freq.min(), 200),
    y = gaus(np.linspace(freq.max(), freq.min(), 200), *popt)-0.2,
    mode = 'lines',
    name = 'lines'
)

plot_data = [trace1, trace2]
plot(plot_data, filename='scatter-mode')

'file://C:\\Users\\pi3\\Documents\\GitHub\\qudi\\scatter-mode.html'

In [6]:
from plotly import __version__
from plotly.offline import init_notebook_mode, plot
from plotly.graph_objs import Scatter

# t=np.linspace(0, 2*4*300e-9, 200)
# y=np.sin(2*np.pi*2.4*1e+6*t)

t=np.linspace(0, 4, 200)
y=np.exp(-t)

trace = Scatter(
    x = t,
    y = y,
    mode = 'lines',
    name = 'lines'
)

plot_data = [trace]
plot(plot_data, filename='scatter-mode')

'file://C:\\Users\\pi3\\Documents\\GitHub\\qudi\\scatter-mode.html'

In [53]:
freq

array([ 2.52525259,  2.51256283,  2.49999997,  2.48756212,  2.47524758,
        2.4630542 ,  2.45098036,  2.43902432,  2.42718452,  2.41545894,
        2.40384612,  2.39234442,  2.38095243,  2.36966825,  2.35849053,
        2.34741776,  2.33644864,  2.3255814 ,  2.31481478,  2.30414754,
        2.29357802])